In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dataset = pd.read_csv('dataset-regression/audi.csv')
dataset

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0
...,...,...,...,...,...,...,...,...,...
10663,A3,2020,16999,Manual,4018,Petrol,145,49.6,1.0
10664,A3,2020,16999,Manual,1978,Petrol,150,49.6,1.0
10665,A3,2020,17199,Manual,609,Petrol,150,49.6,1.0
10666,Q3,2017,19499,Automatic,8646,Petrol,150,47.9,1.4


In [3]:
if "id" in dataset.columns:
    dataset.drop("id",axis=1,inplace=True)
if "date" or "Date"  in dataset.columns:
    if("Date" in dataset.columns):
        dataset.drop("Date",axis=1,inplace=True)
    elif("date" in dataset.columns):
        dataset.drop("date",axis=1,inplace=True)
        

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10668 entries, 0 to 10667
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         10668 non-null  object 
 1   year          10668 non-null  int64  
 2   price         10668 non-null  int64  
 3   transmission  10668 non-null  object 
 4   mileage       10668 non-null  int64  
 5   fuelType      10668 non-null  object 
 6   tax           10668 non-null  int64  
 7   mpg           10668 non-null  float64
 8   engineSize    10668 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 750.2+ KB


In [5]:
datatypes = dataset.dtypes
datatypes

model            object
year              int64
price             int64
transmission     object
mileage           int64
fuelType         object
tax               int64
mpg             float64
engineSize      float64
dtype: object

In [6]:
cat_cols = []
num_cols = []
id1 = "id"
target_column = 'price'
for i in range(len(datatypes)):
    if datatypes.index[i] != target_column:
        if datatypes[i]=='object':
            unqval = dataset[datatypes.index[i]].nunique()
            if (unqval < 300):
                cat_cols.append(datatypes.index[i])
            else:
                del dataset[datatypes.index[i]]
                print('Deleted: ',datatypes.index[i])
        else:
            num_cols.append(datatypes.index[i])
num_cols

['year', 'mileage', 'tax', 'mpg', 'engineSize']

In [7]:
cat_cols

['model', 'transmission', 'fuelType']

In [8]:
cat_data = []
num_data = []
if(len(cat_cols)>0):
    cat_data = dataset[cat_cols]
if(len(num_cols)>0):
    num_data = dataset[num_cols]
y_data = dataset[target_column]

In [9]:
num_data,cat_data

(       year  mileage  tax   mpg  engineSize
 0      2017    15735  150  55.4         1.4
 1      2016    36203   20  64.2         2.0
 2      2016    29946   30  55.4         1.4
 3      2017    25952  145  67.3         2.0
 4      2019     1998  145  49.6         1.0
 ...     ...      ...  ...   ...         ...
 10663  2020     4018  145  49.6         1.0
 10664  2020     1978  150  49.6         1.0
 10665  2020      609  150  49.6         1.0
 10666  2017     8646  150  47.9         1.4
 10667  2016    11855  150  47.9         1.4
 
 [10668 rows x 5 columns],
       model transmission fuelType
 0        A1       Manual   Petrol
 1        A6    Automatic   Diesel
 2        A1       Manual   Petrol
 3        A4    Automatic   Diesel
 4        A3       Manual   Petrol
 ...     ...          ...      ...
 10663    A3       Manual   Petrol
 10664    A3       Manual   Petrol
 10665    A3       Manual   Petrol
 10666    Q3    Automatic   Petrol
 10667    Q3       Manual   Petrol
 
 [10668 r

In [10]:
if len(num_data):
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(missing_values=np.nan, strategy = 'mean')
    imputer.fit(num_data.iloc[:,:].values)
    num_data = pd.DataFrame(imputer.transform(num_data.iloc[:,:].values))
    num_data.columns = num_cols
num_data

,year,mileage,tax,mpg,engineSize
0,2017.0,15735.0,150.0,55.4,1.4
1,2016.0,36203.0,20.0,64.2,2.0
2,2016.0,29946.0,30.0,55.4,1.4
3,2017.0,25952.0,145.0,67.3,2.0
4,2019.0,1998.0,145.0,49.6,1.0
...,...,...,...,...,...
10663,2020.0,4018.0,145.0,49.6,1.0
10664,2020.0,1978.0,150.0,49.6,1.0
10665,2020.0,609.0,150.0,49.6,1.0
10666,2017.0,8646.0,150.0,47.9,1.4


In [11]:
if(len(cat_data)>0 and len(num_data)>0):
    print("1")
    df = pd.concat([num_data,cat_data,y_data],axis=1)
    df = df.dropna().reset_index(drop=True)
    df.isnull().sum()
elif(len(num_data) > 0):
    print("2")
    df = pd.concat([num_data,y_data],axis=1)
    df = df.dropna().reset_index(drop=True)
else:
    print("3")
    df = pd.concat([cat_data,y_data],axis=1)
    df = df.dropna().reset_index(drop=True)
(df)

1


,year,mileage,tax,mpg,engineSize,model,transmission,fuelType,price
0,2017.0,15735.0,150.0,55.4,1.4,A1,Manual,Petrol,12500
1,2016.0,36203.0,20.0,64.2,2.0,A6,Automatic,Diesel,16500
2,2016.0,29946.0,30.0,55.4,1.4,A1,Manual,Petrol,11000
3,2017.0,25952.0,145.0,67.3,2.0,A4,Automatic,Diesel,16800
4,2019.0,1998.0,145.0,49.6,1.0,A3,Manual,Petrol,17300
...,...,...,...,...,...,...,...,...,...
10663,2020.0,4018.0,145.0,49.6,1.0,A3,Manual,Petrol,16999
10664,2020.0,1978.0,150.0,49.6,1.0,A3,Manual,Petrol,16999
10665,2020.0,609.0,150.0,49.6,1.0,A3,Manual,Petrol,17199
10666,2017.0,8646.0,150.0,47.9,1.4,Q3,Automatic,Petrol,19499


In [12]:
cat_data = df[cat_cols]
num_data = df[num_cols]
y_data = df[target_column]
df.drop(target_column,inplace=True,axis=1)
len(y_data)

10668

In [13]:
y_data

0        12500
1        16500
2        11000
3        16800
4        17300
         ...  
10663    16999
10664    16999
10665    17199
10666    19499
10667    15999
Name: price, Length: 10668, dtype: int64

In [14]:
df

,year,mileage,tax,mpg,engineSize,model,transmission,fuelType
0,2017.0,15735.0,150.0,55.4,1.4,A1,Manual,Petrol
1,2016.0,36203.0,20.0,64.2,2.0,A6,Automatic,Diesel
2,2016.0,29946.0,30.0,55.4,1.4,A1,Manual,Petrol
3,2017.0,25952.0,145.0,67.3,2.0,A4,Automatic,Diesel
4,2019.0,1998.0,145.0,49.6,1.0,A3,Manual,Petrol
...,...,...,...,...,...,...,...,...
10663,2020.0,4018.0,145.0,49.6,1.0,A3,Manual,Petrol
10664,2020.0,1978.0,150.0,49.6,1.0,A3,Manual,Petrol
10665,2020.0,609.0,150.0,49.6,1.0,A3,Manual,Petrol
10666,2017.0,8646.0,150.0,47.9,1.4,Q3,Automatic,Petrol


In [15]:
#if len(cat_cols):
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_encoded = pd.DataFrame(OH_encoder.fit_transform(df[cat_cols]))

# One-hot encoding removed index; put it back
X_encoded.index = df.index

# Remove categorical columns (will replace with one-hot encoding)
num_X = df.drop(cat_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X = pd.concat([num_X, X_encoded], axis=1)

In [16]:
OH_X

,year,mileage,tax,mpg,engineSize,0,1,2,3,4,...,22,23,24,25,26,27,28,29,30,31
0,2017.0,15735.0,150.0,55.4,1.4,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2016.0,36203.0,20.0,64.2,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,2016.0,29946.0,30.0,55.4,1.4,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,2017.0,25952.0,145.0,67.3,2.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,2019.0,1998.0,145.0,49.6,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10663,2020.0,4018.0,145.0,49.6,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
10664,2020.0,1978.0,150.0,49.6,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
10665,2020.0,609.0,150.0,49.6,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
10666,2017.0,8646.0,150.0,47.9,1.4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [17]:
count = y_data.value_counts()
count
len(y_data)

10668

In [18]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
OH_X = sc_x.fit_transform(OH_X)

In [19]:
y_data = y_data.iloc[:].values
y_data = y_data.reshape(len(y_data),1)

In [20]:
sc_y = StandardScaler()
y_data = sc_y.fit_transform(y_data)
y_data.shape

(10668, 1)

In [21]:
OH_X.shape

(10668, 37)

In [22]:
OH_X

array([[-0.04644979, -0.3868356 ,  0.35714729, ..., -1.04664348,
        -0.05129892,  1.05216498],
       [-0.50783377,  0.48398914, -1.57832278, ...,  0.95543518,
        -0.05129892, -0.95042129],
       [-0.50783377,  0.21778089, -1.42944047, ..., -1.04664348,
        -0.05129892,  1.05216498],
       ...,
       [ 1.33770214, -1.03038137,  0.35714729, ..., -1.04664348,
        -0.05129892,  1.05216498],
       [-0.04644979, -0.68844184,  0.35714729, ..., -1.04664348,
        -0.05129892,  1.05216498],
       [-0.50783377, -0.55191279,  0.35714729, ..., -1.04664348,
        -0.05129892,  1.05216498]])

In [23]:
y_data

array([[-0.88752136],
       [-0.54605815],
       [-1.01557006],
       ...,
       [-0.48638745],
       [-0.29004611],
       [-0.58882641]])

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train,X_test,y_train,y_test = train_test_split(OH_X,y_data,test_size=0.2,random_state=0)

In [26]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import accuracy_score,confusion_matrix
# nodes = [10,15,20,25,50,100,200]
# accuracy =[]
# for node in nodes:    
#     model = RandomForestRegressor(random_state=0, n_estimators=node)
#     model.fit(X_train,y_train)
#     preds = model.predict(X_test)

In [27]:
#     accs = accuracy_score(y_test,preds)
#     accuracy.append(accs)
#     print(confusion_matrix(y_test,preds),'No of Estimators: ', node,accs)

In [28]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVR()

In [ ]:
from sklearn.model_selection import GridSearchCV
#parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
 #             {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
parameters={
            'C': [0.1, 1, 100, 1000],
            'epsilon': [0.01, 0.1, 1, 5, 10],
            'gamma': [0.005, 0.1, 1, 3, 5]
        }
grid_search = GridSearchCV(estimator = regressor,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

In [ ]:
preds = sc_y.inverse_transform(regressor.predict(X_test))

In [ ]:
preds[:10],sc_y.inverse_transform(y_test)[:10]

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(preds,sc_y.inverse_transform(y_test))

In [ ]:
from xgboost import XGBRegressor   #XGBoostClassifier
classifier = XGBRegressor(max_depth=6)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [ ]:
r2_score(sc_y.inverse_transform(y_pred),sc_y.inverse_transform(y_test))